저상버스의 시간표를 저장하는 DB

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy.sql import select

In [49]:
engine = create_engine("sqlite:///", echo=False)
metadata = MetaData()

Timetable = Table('Timetable', metadata, 
              Column('_id', Integer, primary_key =True, autoincrement=True ),
              Column('routeId', Integer, nullable = False),
              Column('date', String(8), nullable = False), 
              Column('arrivalTm',String(6)),
              Column('posX',String), 
              Column('posY',String))  

metadata.create_all(engine) 
conn = engine.connect()

버스위치api를 반복 호출하면서 현재 운행중인 저상버스가 정류소에 도착하는 시간을 DB에 저장

방법)
- 위치 최초 호출: 운행중인 저상버스의 위치와 정류소 도착 여부 저장(1이면 DB저장)
- 20초 후 재호출: 이미 운행중이던 버스의 도착여부가 0에서 1로 바뀌었다면 DB에 도착시간 저장. 
- 더 이상 운행하는 차량이 없을 때까지 api를 반복 호출하며 각 정류소의 도착 시간을 저장

In [3]:
import requests
from bs4 import BeautifulSoup
import time

##### 기존 코드) 하나의 서비스키, 하나의 노선만 받아올 수 있는 함수

In [50]:
def getData(routeID):
    url =  "http://ws.bus.go.kr/api/rest/buspos/getLowBusPosByRtid"
    serviceKey = "ifKRU73xKAdgp5Z2cQ6BDZSHhEOvKL8TNiEegFTFxtM9AWL6l0Wx8WJ4DfmQDZrTjjb509tqW0hO2wA9FE%2BWsQ%3D%3D"
    data = requests.get(url+ "?serviceKey="+serviceKey+"&busRouteId="+routeID)
    xml = BeautifulSoup(data.text, "lxml")
    busList = xml.findAll("itemlist") #해당 노선의 현재 운행중인 모든 저상버스
    
    return busList


def timetable(routeID):
    busList = getData(routeID)
    bus_stop = {}
    
    while busList: #운행중인 버스가 하나라도 있는 경우
        print(bus_stop.keys())
        for bus in busList:
            vehid = bus.vehid.text
            stop = int(bus.stopflag.text)
            
            print('vehid: ', vehid)
            print('stop: ', stop)
            
            if vehid in bus_stop.keys():
                if not bus_stop[vehid] and stop:  #정류장여부 0->1
                    #DB에 시간, 위치 저장
                    conn.execute(Timetable.insert(), routeId = routeID, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
                    print('db update')
            elif stop == 1:
                    #DB에 시간 저장
                    conn.execute(Timetable.insert(), routeId = routeID, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
                    print('db update')
            bus_stop[vehid] = stop
            
            if int(bus.nextsttm.text) >= int(bus.laststtm.text):  #종점에 도착한 버스는 pop
                bus_stop.pop(vehid)
                
        time.sleep(20)
        busList = getData(routeID)
        
        print("-----------------------------")
        
    

In [51]:
timetable(routeID)

dict_keys([])
vehid:  124031133
stop:  0
vehid:  124031168
stop:  0
vehid:  124031170
stop:  1
db update
vehid:  124031152
stop:  0
vehid:  124031151
stop:  1
db update
vehid:  124031127
stop:  0
vehid:  124031005
stop:  0
vehid:  124031090
stop:  0
vehid:  124031166
stop:  1
db update
vehid:  124031008
stop:  0
vehid:  124031087
stop:  0
vehid:  124031050
stop:  1
db update
vehid:  124031091
stop:  0
vehid:  124031055
stop:  0
-----------------------------
dict_keys(['124031133', '124031168', '124031170', '124031152', '124031151', '124031127', '124031005', '124031090', '124031166', '124031008', '124031087', '124031050', '124031091', '124031055'])
vehid:  124031133
stop:  0
vehid:  124031168
stop:  0
vehid:  124031170
stop:  0
vehid:  124031152
stop:  0
vehid:  124031151
stop:  1
vehid:  124031127
stop:  0
vehid:  124031005
stop:  1
db update
vehid:  124031090
stop:  1
db update
vehid:  124031166
stop:  0
vehid:  124031008
stop:  1
db update
vehid:  124031087
stop:  1
db update
vehid: 

KeyboardInterrupt: 

### 코드 수정 ) 여러 서비스키를 이용하고, 여러 버스노선의 실시간 위치를 한 번에 받아올 수 있음


In [63]:
serviceKey = ["ifKRU73xKAdgp5Z2cQ6BDZSHhEOvKL8TNiEegFTFxtM9AWL6l0Wx8WJ4DfmQDZrTjjb509tqW0hO2wA9FE%2BWsQ%3D%3D"]

def getData(key, routeID):
    url =  "http://ws.bus.go.kr/api/rest/buspos/getLowBusPosByRtid"
    data = requests.get(url+ "?serviceKey="+key+"&busRouteId="+routeID)
    xml = BeautifulSoup(data.text, "lxml")
    busList = xml.findAll("itemlist") #해당 노선의 현재 운행중인 모든 저상버스
    
    return busList


각각의 버스노선을 객체로 만들어 현재 운행중인 해당 노선의 버스 목록과 각 버스의 정류장 도착 여부를 저장함

In [66]:
class LowBus:
    def __init__(self, routeID):
        self.id = routeID
        self.busList = {}
        self.bus_stop = {}

In [74]:
lowbusID = ['100100055', '100100549' ]
lowbusList= []

for lowbus in lowbusID:
    lowbusList.append(LowBus(lowbus))   #전체 lowbus 객체 생성


for key in serviceKey:

#     call =  10000 // len(lowbusID)
    call = 3
    while call > 0 :  #call 수 만큼 반복 가능 - 할당된 call을 다 사용하면 다음 서비스키 사용
        
        for lowbus in lowbusList:
            print(lowbus.id)
            lowbus.busList = getData(key, lowbus.id)
            call -= 1
            
            if lowbus.busList:
                for bus in lowbus.busList:
                    vehid = bus.vehid.text
                    stop = int(bus.stopflag.text)
                    print('vehid: ', vehid)
                    print('stop: ', stop)
            
                    if vehid in lowbus.bus_stop.keys():
                        if not lowbus.bus_stop[vehid] and stop:  #정류장여부 0->1
                            #DB에 시간, 위치 저장
                            conn.execute(Timetable.insert(), routeId = lowbus.id, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
                            print('db update')
                    elif stop == 1:
                            #DB에 시간 저장
                            conn.execute(Timetable.insert(), routeId = lowbus.id, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
                            print('db update')
                    lowbus.bus_stop[vehid] = stop

                    if vehid in lowbus.bus_stop.keys() and (int(bus.nextsttm.text) >= int(bus.laststtm.text)):  #종점에 도착한 버스는 pop
                        lowbus.bus_stop.pop(vehid)
            
            if call <= 0:
                break
                
            print("-----------------------------")      
              
        time.sleep(20)       

        

100100055
vehid:  124031125
stop:  0
vehid:  124031133
stop:  1
db update
vehid:  124031168
stop:  0
-----------------------------
100100549
vehid:  110056205
stop:  1
db update
vehid:  110056313
stop:  0
vehid:  110056012
stop:  1
db update
vehid:  110056009
stop:  0
vehid:  110056260
stop:  1
db update
vehid:  110056212
stop:  0
-----------------------------
100100055
vehid:  124031125
stop:  1
db update
vehid:  124031133
stop:  1
vehid:  124031168
stop:  1
db update


In [72]:
for row in conn.execute(select([Timetable])):
    print(row)

(1, 100100055, '20180815', '150548', '127.127486', '37.550329')
(2, 100100055, '20180815', '150542', '127.085723', '37.50511')
(3, 100100055, '20180815', '150556', '127.068351', '37.501211')
(4, 100100055, '20180815', '150556', '127.124027', '37.540367')
(5, 100100055, '20180815', '150603', '127.02771', '37.489862')
(6, 100100055, '20180815', '150609', '127.032876', '37.489747')
(7, 100100055, '20180815', '150611', '127.085007', '37.505174')
(8, 100100055, '20180815', '150611', '127.119885', '37.529152')
(9, 100100055, '20180815', '150607', '127.143032', '37.554657')
(10, 100100055, '20180815', '150628', '127.164429', '37.556565')
(11, 100100055, '20180815', '150633', '127.14698', '37.554758')
(12, 100100055, '20180815', '150627', '127.047924', '37.494666')
(13, 100100055, '20180816', '001850', '127.068333', '37.501046')
(14, 100100055, '20180816', '001841', '127.127506', '37.549966')
(15, 100100055, '20180816', '001901', '127.118607', '37.526705')
(16, 100100055, '20180816', '001935',

### 간단한 코드) 호출했을 때 버스가 정류장에 도착해 있다면 DB에 저장 
##### 호출 시간 간격이 길기 때문에(60초) 이전 호출에서 해당 버스가 정류장에 있었는지 여부와 상관없음

In [75]:
lowbusID = ['100100055', '100100549' ]
lowbusList= []

for lowbus in lowbusID:
    lowbusList.append(LowBus(lowbus))   #전체 lowbus 객체 생성


for key in serviceKey:

#     call =  10000 // len(lowbusID)
    call = 4
    while call > 0 :  #call 수 만큼 반복 가능 - 할당된 call을 다 사용하면 다음 서비스키 사용
        
        for lowbus in lowbusList:
            print(lowbus.id)
            lowbus.busList = getData(key, lowbus.id)
            call -= 1
            
            if lowbus.busList:
                for bus in lowbus.busList:
                    vehid = bus.vehid.text
                    stop = int(bus.stopflag.text)
                    print('vehid: ', vehid)
                    print('stop: ', stop)
            
                    if stop == 1:
                        #DB에 시간 저장
                        conn.execute(Timetable.insert(), routeId = lowbus.id, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
                        print('db update')
            
            if call <= 0:
                break
                
            print("-----------------------------")      
              
        time.sleep(60)       

        

100100055
vehid:  124031125
stop:  0
vehid:  124031133
stop:  1
db update
-----------------------------
100100549
vehid:  110056205
stop:  0
vehid:  110056313
stop:  0
vehid:  110056012
stop:  1
db update
vehid:  110056009
stop:  1
db update
-----------------------------
100100055
vehid:  124031125
stop:  0
vehid:  124031133
stop:  1
db update
-----------------------------
100100549
vehid:  110056205
stop:  0
vehid:  110056313
stop:  0
vehid:  110056012
stop:  0
vehid:  110056009
stop:  0
